## [ Tutorial ] 
---
### [ 0. Airflow 란? ]
> 빅 데이타를 처리하다보면 여러 개의 task로 나뉘어 순차적으로 실행할 필요가 있다.  
> 배치 작업처럼, 특정 시간마다 task들을 순차적으로 실행시켜줄 무언가가 필요하다.  
> 하둡에서 워크플로우를 관리해주는 것이 `Airflow`이다  
> 또한, Airflow는 파이썬으로 코딩하기 편할 뿐더러, `병렬성`을 가지고있다.  

### [ 1. Example Pipeline Definition ]
- 아래의 코드는 `basic pipeline definition`의 예제이다.


In [ ]:
from datetime import timedelta

# DAG 객체 for instantiating DAG
from airflow import DAG
# BashOperator는 Operator들이 실행하기 위해 필요하다.
from airflow.operators.bash_operator import BaseOperator
from airflow.utils.dates import days_ago
# 각 args는 Operator들에게 전달된다.
default_args = {
    'owner' : 'airflow',
    'depends_on_past' : False,
    'start_date' : days_ago(2),
    'email' : ['airflow@example.com'],
    'email_on_failure' : False,
    'email_on_retry' : False,
    'retries' : 1,
    'retry_delay' : timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
    # 'wait_for_downstream': False,
    # 'dag': dag,
    # 'sla': timedelta(hours=2),
    # 'execution_timeout': timedelta(seconds=300),
    # 'on_failure_callback': some_function,
    # 'on_success_callback': some_other_function,
    # 'on_retry_callback': another_function,
    # 'sla_miss_callback': yet_another_function,
    # 'trigger_rule': 'all_success'
}


dag = DAG(
    'tutorial',
    default_args=default_args,
    description='A simple tutorial DAG',
    schedule_interval=timedelta(days=1),
    )

t1 = BashOperator(
    task_id = 'print_date',
    bash_command='date',
    dag=dag,
)

t2 = BashOperator(
    task_id = 'sleep',
    depends_on_past = False,
    bash_command='sleep 5',
    retries=3,
    dag=dag
)

dag.doc_md = __doc__

t1.doc_md = """\
#### Task Documentation
You can document your task using the attributes `doc_md` (markdown),
`doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
rendered in the UI's Task Instance Details page.
![img](http://montcs.bloomu.edu/~bobmon/Semesters/2012-01/491/import%20soul.png)
"""
templated_command = """
{% for i in range(5) %}
    echo "{{ ds }}"
    echo "{{ macros.ds_add(ds, 7)}}"
    echo "{{ params.my_param }}"
{% endfor %}
"""

t3 = BashOperator(
    task_id='templated',
    depends_on_past=False,
    bash_command=templated_command,
    params={'my_param': 'Parameter I passed in'},
    dag=dag,
)

t1 >> [t2, t3]

### [ 2. DAG Definition file ]
- 위의 Airflow Python script는 DAG 구조의 configuration을 나타내는 코드에 해당한다.
    - `DAG`는 Airflow에서 task들의 워크플로우를 관리해준다.
    - `DAG`는 task간의 스캐줄을 관리할 뿐, task 자체를 DAG에 명시하지 않는다.


- DAG 객체를 정의하기 위해 사용되는 scirpt이다.
- scheduler가 변화를주기적으로 반영하기위해서 계속 실행하기 때문에 이것은 빠르게 evaluate될 필요가있다.


- Different tasks run on different workers at different points in time
    - 즉 다수의 사람들이 동시에 쓰는 것은 불가능하다.
    - 하지만, task1이 수행한 결과를 이용해서 task2를 수행해야할 경우가 있다.  
    물론 중간에 다른 저장소를 이용할 수도 있지만,  
    간단한 결과라면 Airflow에서 제공하는 `XCOM`이라는 메모리 저장소를 이용해서 결과를 주고받을 수 있다.  
    
    

### [ 3. Importing Modules ] 
- `Airflow Pipeline`은 `Airflow DAG`객체를 정의하기 위해서 `Python` script를 사용한다.
- `Airflow`를 사용하기위해 필요한 라이브러리들은 다음과 같다.

In [ ]:
from datetime import timedelta

# DAG 객체를 instantiate하기 위해 필요하다.
from airflow import DAG

# BashOperator는 Operator가 실행되기 위해 필요하다.
from airflow.operators.bash_operator import BaseOperator
from airflow.utils.dates import days_ago

### [ 4. Default Arguments ]
- DAG와 몇 가지 task를 만들 때, DAG에서 사용할 `Default Arg`를 지정할 수 있다.  
    - 모든 task가 공유 받을 수 있다.
    - 매개변수가 사전형태로 정의된다.
    - 환경, 목적에 따라 필요한 key들이 모두 다르다.
    - 참조 : http://airflow.apache.org/docs/stable/_api/airflow/models/index.html#airflow.models.BaseOperator

In [ ]:
# These args will get passed on to each operator
# You can override them on a per-task basis during operator initialization
default_args = {
    'owner' : 'airflow',                                   
    'depends_on_past' : False,
    'start_date' : days_ago(2),
    'email' : ['airflow@example.com'],
    'email_on_failure' : False,
    'email_on_retry' : False,
    'retries' : 1,
    'retry_delay' : timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
    # 'wait_for_downstream': False,
    # 'dag': dag,
    # 'sla': timedelta(hours=2),
    # 'execution_timeout': timedelta(seconds=300),
    # 'on_failure_callback': some_function,
    # 'on_success_callback': some_other_function,
    # 'on_retry_callback': another_function,
    # 'sla_miss_callback': yet_another_function,
    # 'trigger_rule': 'all_success'
}

### [ 5. Instantiate a DAG ]
- task를 넣기위해서는 `DAG` 객체가 필요하다.


- DAG에 unique identifier를 제공하기위한 `dag_id`
    - dag_id는 생략하여 사용할 수 있다.  
        아래의 예제 같은 경우 dag_id = 'tutorial'을 단순히 'tutorial'로만 표기할 수 있다.


- `default_args` : 위에서 정의한 default arguements dict를 dag에 넘겨준다.  


- `schedule_interval` : 스캐줄 인터버를 넘겨준다.
    - python `datetime` library로 표현가능하다. (아래의 예제에서는 인터벌이 하루라는 의미이다.
    - `schedule_interval=None`, `schedule_interval='@once` 등으로도 작성 가능하다.


- 참조 : https://airflow.apache.org/docs/stable/_api/airflow/models/dag/index.html

In [ ]:
dag = DAG(
    dag_id = 'tutorial',
    default_args=default_args,
    description='A simple tutorial DAG',
    schedule_interval=timedelta(days=1),
)

### [ 6. Tasks ] 
- `Task`는 operator 객체가 instantiate했을 때 만들어진다.
    - 이처럼, operator로부터 instantiated된 객체들(예제에서는 t1, t2)을 `constructor`이라 지칭한다.


- Task는 무조건 `task_id`와 `owner`를 포함해야한다! 
    - `task_Id` : unique identifier for the task
    - 포함안될 경우, airflow will raise an exception. (??...)
 
 
- 각 Task는 DAG 객체를 정의할 때 사용했던 `default argument`를 상속받는다.
    -  하지만, default argument가 지정되어도, Task안에 따로 argument가 언급되어 있을 경우 default arguement는 무시된다.
    - argument의 우선순위는 차례대로 아래와 같다.
        - 1. 명시적으로 지정된 arguments
        - 2. default_argument에서 지정된 arguments
        - 3. operator의 default value로 지정된 arguments ( 이건 operator마다 있을 수도 없을 수도 있다. )

In [ ]:
t1 = BashOperator(
    task_id='print_date',
    bash_command='date',       # 아래와 달리 이 task는 retries =1 이 적용된다.
    dag=dag,
)

t2 = BashOperator(
    task_id='sleep',
    depends_on_past=False,
    bash_command='sleep 5',
    retries=3,                 # default arguments에서 retries=1이라했지만, 따로 task안에 명시했으므로 얘는 retries=3이 적용된다.
    dag=dag,
)

### [ 7. Templating with Jinja - 이해 부족 ]
- Jinja 템플릿 방식 적용이 가능하다.
    - 매개변수를 {{ dsfdsf }} 식으로 넣어서 사용이 가능하다.
    
> 아직 Jinja에 대한 개념이 부족해서 무슨 말인지 이해를 잘 못하겠다.

In [ ]:
templated_command = """
{% for i in range(5) %}
    echo "{{ ds }}"
    echo "{{ macros.ds_add(ds, 7)}}"
    echo "{{ params.my_param }}"
{% endfor %}
"""

t3 = BashOperator(
    task_id='templated',
    depends_on_past=False,
    bash_command=templated_command,
    params={'my_param': 'Parameter I passed in'},
    dag=dag,
)

### [ 8. Adding DAG and documentation ]
- DAG나 각각의 Task에 설명을 달 수 있다.
    - DAG documentation : 형식은 markdown을 지원한다.
    - Task documentation : 형식은 plain text, markdown, reStructuredText, json, yaml 지원한다.

In [ ]:
dag.doc_md = __doc__

t1.doc_md = """\
#### Task Documentation
You can document your task using the attributes `doc_md` (markdown),
`doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
rendered in the UI's Task Instance Details page.
![img](http://montcs.bloomu.edu/~bobmon/Semesters/2012-01/491/import%20soul.png)
"""

### [ 9. Setting up Dependencies ]
- 각 task(예제에서는 t1, t2, t3)의 종속관계, 즉 실행 순서등을 입력할 수 있다.

<img width="500" alt="dependencies_visualization" src="https://user-images.githubusercontent.com/53929665/96223731-5abb4e80-0fc9-11eb-8ed2-54768b860861.PNG">


- 또한, task의 list를 이용하여 표현할 수 있다.
    - `t1.set_downstream([t2, t3])` | `t1 >> [t2,t3]` | `[t2, t3] << t1`   
    : 위의 표현은 모두 t1 작업이 끝난 후에 t2,t3작업을 수행하라는 의미이다.

In [ ]:
t1.set_downstream(t2)

# This means that t2 will depend on t1
# running successfully to run.
# It is equivalent to:
t2.set_upstream(t1)

# The bit shift operator can also be
# used to chain operations:
t1 >> t2

# And the upstream dependency with the
# bit shift operator:
t2 << t1

# Chaining multiple dependencies becomes
# concise with the bit shift operator:
t1 >> t2 >> t3

# A list of tasks can also be set as
# dependencies. These operations
# all have the same effect:
t1.set_downstream([t2, t3])
t1 >> [t2, t3]
[t2, t3] << t1

### [ 10. Testing ]

#### (1) Running the Script 
- 만약 위의 예제 코드를 `tutorial.py`로 `example_dags`라는 임시 DAG폴더에 저장하는 것을 가정한다.  
(일반적인 `DAG` 폴더의 위치는 `~/airflow/dags`에 위치한다.

- 테스트 실행은 CMD또는 SHELL에서 다음과 같이 일반적인 파이썬 스크립트의 실행과 동일하다.
```python
python ~/airflow/example_dags/tutorial.py
```



#### (2) Command Line Metadata Validation 
다음 스크립트들을 실행할 경우 현재 script의 dag에 관한 status를 확인할 수 있다.


> ```python
> airflow list_dags
> ```


- tutorial이 정상적으로 active하는 것을 확인할 수 있다.
    
    
![2](https://user-images.githubusercontent.com/53929665/96226451-9c4df880-0fcd-11eb-8d39-20e1eea033e1.png)

</br>
</br>


> ```python
> airflow list_tasks tutorial
> ```


- tutorial이라는 dag의 task list가 모두 출력되는 것을 확인할 수 있다.


![3](https://user-images.githubusercontent.com/53929665/96226454-9d7f2580-0fcd-11eb-8d82-e0e65358a635.png)

</br>
</br>



> ```python
> airflow list_tasks tutorial --tree
> ```


- tutorial DAG안에 있는 task들을 `tree형 구조`로 볼 수 있다.
    - <Task(BashOperator): print_date>       →  1순위
        - <Task(BashOperator): sleep>        →  공동 2순위 
        - <Task(BashOperator): templated>    →  공동 2순위
 

![4](https://user-images.githubusercontent.com/53929665/96226455-9d7f2580-0fcd-11eb-9c4c-99bcc3b04b06.png)






#### (3) 특정일 기준의 actual task instance

- python script 자체를 실행하여 테스트할 수 있지만, 다음과 같이 task 단위로도 테스트가 가능하다.
- 날짜를 지정하는 것을 context내에서는 `execution_date`라고 부른다.

 
- 기본형식 :
```python 
airflow test [dag_id] [task_id] [excution_date]
```
 
- 위 예제 같은 경우,
```python
airflow test tutorial print_date 2020-10-16
```

<img width="500" alt="1" src="https://user-images.githubusercontent.com/53929665/96224884-33fe1780-0fcb-11eb-86d4-6a3a0ab21a97.PNG">

- 마지막으로 `return code 0`가 출력되면 정상 종료되었다는 의미이다.
- 과거 시간으로 동작하지만, 미래 시간대로 설정하면 동작하지 않는다.
- Airflow에서는 한국 시간보다 9시간 느린 UTC 표준시간시를 사용한다.

### [ 11. Operation ]
- airflow는 다음의 쉘 명령어 두 개로 실행 가능하다.
```shell
airflow webserver
airflow scheduler
```
    - `Airflow Webserver`는 웹 UI를 표현하고, workflow 상태 표시 및 실행, 재시작, 수동 조작, 로그 확인 등 가능
    - `Airflow Scheduler`는  
    작업 기준이 충족되는지 여부를 확인  
    종속 작업이 성공적으로 완료되었고, 예약 간격이 주어지면 실행할 수 있는 직업인지, 실행 조건이 충족되는지 등  
    의 충족 여부가 DB에 기록되면, task들이 worker에게 선택되서 작업을 실행한다.
    
    
- [ 10. test ] 에서 test를 통해 DAG의 동작을 확인했다면 스케줄에 맞춰 실행시키면 된다.
- 아래와 같이 localhos:8080 web에서 DAG 상태를 On으로 바꿔주면된다.

<img width="666" alt="5" src="https://user-images.githubusercontent.com/53929665/96229621-27c98880-0fd2-11eb-9a12-2b0c726f90c0.PNG">


- 16일부터 19일까지 실행시킨 결과  
DAG Runs에서 '17일 00:00:00', '18일 00:00:00' scheduler대로 자동 실행된 것을 확인할 수 있다.  
(scheduler를 제외한 manual은 수동으로 trigger 한 것이다.)

<img width="717" alt="6" src="https://user-images.githubusercontent.com/53929665/96404177-381d7580-1215-11eb-8d8b-966e6de20367.PNG">



   > (Detail)
   > - 실행날짜가 10월 16일 오후이므로, 17일 00:00:00시에 정상적으로 첫 수행이 시작되었다.
   > - Day단위로 수행되는 tutorial DAG가 17일 18일 총 2일치의 작업 수행이 확인되었다.
   > - 지금 이 문서를 작성하는 시간은 10월 19일 00:00:00이 지났지만, 10월19일 수행기록은 확인되지않는다.

- 10월 19일에는 왜 tutorial DAG가 실행되지 않았을까?
    - 왜냐하면 Trigger은 해당 기간이 끝난뒤에 진행되기 때문이다.
    - 일 단위로 수행하는 tutorial DAG는 `00시00분 ~ 23시59분` 까지가 한 기간에 해당한다.
    - 따라서, 오늘 날짜인 10월 19일의 기간이 아직 종료되지 않았기에 작업이 진행되지 않았다.
    - ex) 10월 19일같은 경우, DAG는 `10월19일00:00`에 execution해서 `10월19일23:59` 직후에 자동으로 Trigger된다.

---
#### index
- moons08's github.io    : https://moons08.github.io/programming/airflow-example/
- zzsza's github.io      : https://zzsza.github.io/data/2018/01/04/airflow-1/
- airflow tutorial guide : http://airflow.apache.org/docs/stable/tutorial.html